

###Ahmad Aljarf 

###Next Word Prediction

###LetsGrowMore

###Batch: January 2023

import the required libraries

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

uploading the file

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 1661-0.txt to 1661-0.txt


Now load the dataset:

In [3]:
text = open('1661-0.txt').read().lower()
print('corpus length:', len(text))

corpus length: 581888


Split the entire dataset into each word in order without the presence of special characters:

In [4]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

For the feature engineering part, lets make the unique sorted words list. then need to create a dictionary with each word form so, the unique_words list as key and its corresponding position as value:

In [5]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work:

In [6]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


Lets create two numpy array:
 X (to store the features) and Y(for store the corresponding label):

In [7]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

We iterate X and Y if the word is present then the corresponding position is made 1:

In [8]:
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

Lets check a single sequence:

In [9]:
print(X[0][0])

[False False False ... False False False]


Now lets build the model as the following:
a single-layer LSTM model with 128 neurons, a fully connected layer, and a softmax function for activation:

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

Training the model:
The model will be trained with 5 epochs with an RMSprop optimizer and the learning rate is 0.01:

In [25]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=5, shuffle=True).history

Epoch 1/5
811/811 [==============================] - 169s 207ms/step - loss: 3.5865 - accuracy: 0.5070 - val_loss: 9.7869 - val_accuracy: 0.0765
Epoch 2/5
811/811 [==============================] - 169s 208ms/step - loss: 3.4450 - accuracy: 0.5295 - val_loss: 9.9371 - val_accuracy: 0.0736
Epoch 3/5
811/811 [==============================] - 170s 209ms/step - loss: 3.3914 - accuracy: 0.5431 - val_loss: 10.1678 - val_accuracy: 0.0672
Epoch 4/5
811/811 [==============================] - 170s 210ms/step - loss: 3.3387 - accuracy: 0.5578 - val_loss: 10.2437 - val_accuracy: 0.0677
Epoch 5/5
811/811 [==============================] - 171s 210ms/step - loss: 3.2732 - accuracy: 0.5711 - val_loss: 10.4152 - val_accuracy: 0.0663


After training model lets save the trained model and just load it back as needed:

In [32]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))

model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

Prediction:

Now we use the model to predict new words.To do that we input the sample as a feature vector. we convert the input string to a single feature vector:

In [33]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

To choose the best possible n words after the prediction from the model is done by sample function:

In [34]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)
                           

Eventually, for prediction, lets apply the function predict_completions which uses the model to predict and return the list of n predicted words:

In [35]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

Now lets see how it predicts, we apply tokenizer.tokenize to remove the punctuations and also we choose 5 first words because our predicts baseed on 5 previous words:

In [37]:
q =  "You can be a great one never give up"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  You can be a great one never give up
Sequence:  you can be a great
you
can
be
a
great
next possible words:  ['her', 'well', 'back', 'be', 'mr']
